In [ ]:
### create netcdf of full 01/01/2010 - 31/12/2020 period, with NaNs where data is missing, for each satellite
### these netcdfs are the ones used for the analysis in this repository

import numpy as np
import datetime
import tqdm
import os
import glob
import netCDF4 as nc4
import xarray as xr

# load auxiliary
lats = np.load('/home/cjn/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
lons = np.load('/home/cjn/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
datapath = '/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/daily_numpys/'

sats = ['CS2_CPOM', 'CS2_LARM', 'CS2S3_CPOM', 'CS2S3_LARM', 'AK_CPOM', 'AK_LARM', 'IS2']

for sat in sats:
    location = datapath + sat + '/'
    days = np.arange(0,4018)
    dates = [datetime.date(2010,1,1)+datetime.timedelta(days=int(days)) for days in days]

    fb = np.full((len(days),360,360), np.nan)
    unc = np.full((len(days),360,360), np.nan)
    files = sorted(glob.glob(location+'*.npy'))

    if len(files)>0:
        for day in tqdm.tqdm(days):
            date = dates[day].strftime('%Y%m%d')
            for f in os.listdir(location):
                if 'FB_interp' in f:
                    if date in f:
                        fb[day] = np.load(location+str(f))
                    else:
                        pass
                if 'FB_uncertainty' in f:
                    if date in f:
                        unc[day] = np.load(location+str(f))
                    else:
                        pass

        # create NETCDF
        if sat == 'IS2':
            data_vars = {'Laser Freeboard':(['t','x','y'],fb),
                        'Uncertainty':(['t','x','y'],unc)}
            ds = xr.Dataset(data_vars = data_vars,
                            coords={'Longitude':(['x','y'],lons),
                                    'Latitude':(['x','y'],lats),
                                     'Day':(['t'],days)}     )
            ds.attrs = {'Description': "daily-resolution interpolated laser freeboard data on 25km EASE-Grid 2.0 (Nab et al. 2022)",
                        'Laser Freeboard': "daily-resolution laser freeboard (m)",
                        'Uncertainty': "daily-resolution laser freeboard uncertainty (m)",
                        'Day': "day since 1 January 2010"}
            ds.to_netcdf(f'/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/NETCDFs/'+sat+'_full.nc','w')
        else:

            data_vars = {'Radar Freeboard':(['t','x','y'],fb),
                        'Uncertainty':(['t','x','y'],unc)}
            ds = xr.Dataset(data_vars = data_vars,
                            coords={'Longitude':(['x','y'],lons),
                                    'Latitude':(['x','y'],lats),
                                     'Day':(['t'],days)}     )
            ds.attrs = {'Description': "daily-resolution interpolated radar freeboard data on 25km EASE-Grid 2.0 (Nab et al. 2022)",
                        'Radar Freeboard': "daily-resolution radar freeboard (m)",
                        'Uncertainty': "daily-resolution radar freeboard uncertainty (m)",
                        'Day': "day since 1 January 2010"}
            ds.to_netcdf(f'/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/NETCDFs/'+sat+'_full.nc','w')

    else: pass